# OBIS full export

Sometimes you need to do analyses using big chunks of OBIS data. In those cases, downloading it through the API is not practical. OBIS also offer the full export of all OBIS occurrence data in `parquet` or `csv` formats.

You can download the full export here (~15GB): https://obis.org/data/access/

We will work with the `parquet` file. [Parquet](https://parquet.apache.org/) is a column-based performant and light format developed by Apache. We can work with `parquet` files on R and Python using the `arrow` [package.](https://arrow.apache.org/docs/r/index.html). You can learn more about parquet in [this tutorial](https://resources.obis.org/tutorials/arrow-obis/).

Loading all this data at once in your computer is usually impossible, as it will not fit in memory. But that is not a problem! Arrow offer us many ways of loading just the structure of the file and perform filters without loading the dataset in memory. Let's explore how to do that.

<span style="color: orange;">NOTE: instead of downloading all the data, we will use the S3 file in this practical.</span>